In [7]:
import fitz
import pandas as pd
import re

In [9]:
#Check if a word pair is present in the page. Input: Page text as a list and one pair of words. Output: True or False
def find_word(word_list, pair1):
    pair1_found = False

    for i in range(len(word_list) - 1):
        if word_list[i] == pair1[0] and word_list[i + 1] == pair1[1]:
            pair1_found = True

    return pair1_found

In [10]:
#Check if two word pairs are present in the page. Input: Page text as a list and two pairs of words. Output: True or False
def find_word_pairs(word_list, pair1, pair2):
    pair1_found = False
    pair2_found = False
    
    # Convert all words and pairs to lowercase for case-insensitive comparison
    word_list_lower = [item.lower() for item in word_list]
    pair1_lower = (pair1[0].lower(), pair1[1].lower())
    pair2_lower = (pair2[0].lower(), pair2[1].lower())
    
    for item in word_list_lower:
        if item.startswith(pair1_lower[0]) and any(word.startswith(pair1_lower[1]) for word in word_list_lower):
            pair1_found = True
        if item.startswith(pair2_lower[0]) and any(word.startswith(pair2_lower[1]) for word in word_list_lower):
            pair2_found = True
        
    return pair1_found, pair2_found

In [11]:
#Merge -ve sign which are split from value. Input: Page text as a list. Output: Page text as a list after attaching -ve sign with following numeric value
def merge_dash(data):
    result = []
    skip_next = False

    for i in range(len(data)):
        if skip_next:
            skip_next = False
            continue

        if data[i] == '-' and i + 1 < len(data) and re.match(r'^[0-9\.]', data[i + 1]):
            result.append(data[i] + data[i + 1])
            skip_next = True
        else:
            result.append(data[i])

    return result

In [12]:
# Merge single digit (not 0) which are split from numeric values. Input: Page text as a list. Output: Page text as a list after attaching single digit with preceding numeric values. 
def merge_single_digit(data):
    output = []

    i = 0
    while i < len(data):
        if i < len(data) - 1 and data[i + 1].isdigit() and len(data[i + 1]) == 1 and data[i + 1] != '0':
            if data[i].replace('.', '', 1).isdigit() or 'E' in data[i]:
                output.append(data[i] + data[i + 1])
            else:
                output.append(data[i])
                output.append(data[i + 1])
            i += 2
        else:
            output.append(data[i])
            i += 1
    return output

In [13]:
# Merge numerical values that are split at "-", "+" or "E". Input: Page text as a list. Output: Page text as a list after attaching numeric values that are split. 
def merge_numericals(data):
    merged_data = []
    for sublist in data:
        merged_sublist = []
        skip_next = False
        for i in range(len(sublist) - 1):
            if skip_next:
                skip_next = False
                continue
            if (sublist[i].endswith(("-", "+", "E"))) and (sublist[i+1][0].isdigit()):
                merged_sublist.append(sublist[i] + sublist[i+1])
                skip_next = True
            else:
                merged_sublist.append(sublist[i])
        if not skip_next:
            merged_sublist.append(sublist[-1])  # Add the last element if it wasn't merged
        merged_data.append(merged_sublist)
    return merged_data

In [14]:
#Check if a value is numeric. Input: A string element from a list. Output: True or False
def is_numeric(s):
    try:
        # Replace commas with periods to handle decimal and scientific notation
        s = s.replace(',', '.')
        float(s)
        return True
    except ValueError:
        return False

In [15]:
#Merge superscript/Subscript with indicator. Input: Page text as a list. Output: Page text as a list after attaching super/sub script with indicator.
def attach_single_digit(lines):
    for sublist in lines:
        for i in range(1, len(sublist)):
            if sublist[i].isdigit() and len(sublist[i]) == 1 and sublist[i] != '0':
                sublist[i - 1] += sublist[i]
                sublist.pop(i)
                break
    return lines

In [16]:
#Extract the table as a list handling empty spaces, split -ve values and split single digits. Input: Document, Page number and start_keyword. Output: Table as a list with no empty spaces, no split -ve values and no split single digits.   
def extract_text_from_page(document, page_num, start_keyword):
    page = document.load_page(page_num)
    text = page.get_text()
    sections = re.split(r'\n\n+', text.strip())
    lines = sections[0].split('\n')
    
    # Update to check if the line starts with the start_keyword
    pos = next((i for i, line in enumerate(lines) if line.strip().lower().startswith(start_keyword.lower())), None)
    
    if pos is not None:
        lines = lines[pos:]
        
    lines = [val for val in lines if val.strip()]
    lines = [item for sublist in [element.split() for element in lines] for item in sublist]
    lines = merge_dash(lines)
    lines = merge_single_digit(lines)
  
    return lines

In [17]:
#Process the table into rows (Attach indicators and group them with their 15 values). Input: Table as a list with split indicators. Output: List of lists, where each list is a row in the table. 
def parse_lines_to_result(lines, ind_rows, standard_rows):
    result = []
    temp = []

    # Attach indicators by attaching non numeric elements
    for item in lines:
        if is_numeric(item):
            if len(item) == 1 and item != '0':  # Check if the numeric item is a single digit and not 0
                temp.append(item)  # Include it in the current non-numeric group
            else:
                if temp:
                    result.append(" ".join(temp))
                    temp = []
                result.append(item)
        else:
            temp.append(item)
    
    # If there are any remaining items in temp, join and add them to the result
    if temp:
        result.append(" ".join(temp))
  
    #Standardize the indicators using master dictionary
    for i in range(len(result)):
        for standard_row in standard_rows:
            if result[i] in standard_row:
                result[i] = standard_row[0]
                break

    #Group indicator and 15 values into rows
    rows = []
    i = 0
    while i < len(result):
        
        current_item = result[i]
        
        if any(current_item.lower().startswith(row.lower()) for row in ind_rows):
            row = []
            for j in range(i, i + 16):
                if j < len(result):
                    item = result[j]
                    
                    #Check if any of the item is an indicator which means 15 values are not available
                    if any(item.lower().startswith(ind_row.lower()) for ind_row in ind_rows) and j > i:
                        break
                    row.append(item)
                else:
                    row.append('0')
            # Fill the remaining items with '0' to make the row length 16
            row.extend(['0'] * (16 - len(row)))
            rows.append(row)
            i = j  # Update the index to the last checked position
        else:
            i += 1

    return rows

In [18]:
#Extract values between two keywords from a list. Input: Page text as a list, start and end keyword. Output: String that attached every element between start and end keywords
def extract_values_between_keywords(lines, start_keyword, end_keyword):
    start_idx = -1
    end_idx = -1

    # Find the start index
    for i, line in enumerate(lines):
        if line.strip().endswith(start_keyword):
            start_idx = i + 1
            break

    # Find the end index
    for i, line in enumerate(lines[start_idx:], start=start_idx):
        if line.strip().startswith(end_keyword):
            end_idx = i
            break

    # Extract and return the values between the keywords
    if start_idx != -1 and end_idx != -1:
        return " ".join(lines[start_idx:end_idx]).strip()
    else:
        return ""

In [19]:
#(Handle exception case): Format product name where it is stored as image. Input: Product name without "Orea" (image) . Output: "Orea" added to product name
def add_orea_to_thickness(prod_name):
    # Regular expression to match if the product name starts with thickness (e.g., '10 mm')
    match = re.match(r'(\d+\s*mm)(.*)', prod_name, re.IGNORECASE)
    
    if match:
        thickness = match.group(1).strip()
        rest_of_name = match.group(2).strip()
        # Add 'Orea' before the thickness
        return f'Orea {thickness} {rest_of_name}'
    else:
        return prod_name

In [20]:
#Initialise all the variables.
#Standard rows contains a list of lists where each list contains all the variations of a particular indicator.
#env_rows, resource_rows, waste_rows are the variations or particular indiactor for that specific table.
#Two pairs of keywords for locating the table pages. 
# Columns for each table
#Keyword to locate the start of each table. 

file_path = 'Dictionary.xlsx'  
df = pd.read_excel(file_path)

# Convert each column to a list including the header
data_with_headers = [[col] + df[col].tolist() for col in df.columns]

standard_rows = [[x for x in sublist if not (pd.isna(x) or (isinstance(x, float) and math.isnan(x)))] for sublist in data_with_headers]

standard_rows = standard_rows[1::]

env_rows = standard_rows[:13]
resource_rows = standard_rows[13:23]
waste_rows = standard_rows[23:]

env_rows = [item for sublist in env_rows for item in sublist]
resource_rows = [item for sublist in resource_rows for item in sublist]
waste_rows = [item for sublist in waste_rows for item in sublist]


# Parameters for Environmental Impacts
env_impacts_pair1 = ("environmental", "impacts")
env_impacts_pair2 = ("climate", "change")
env_impacts_columns = ['Environmental indicators', 'A1 / A2 / A3', 'A4 Transport', 'A5 Installation', 'B1 Use', 'B2 Maintenance', 'B3 Repair', 'B4 Replacement', 'B5 Refurbishment', 'B6 Operational energy use', 'B7 Operational water use', 'C1 Deconstruction / demolition', 'C2 Transport', 'C3 Waste processing', 'C4 Disposal', 'D Reuse, recovery, recycling']
env_impacts_start_keyword = "Climate Change"
env_impacts_num_columns = len(env_impacts_columns)

# Parameters for Resource Use
resource_use_pair1 = ("resources", "use")
resource_use_pair2 = ("primary", "energy")
resource_use_columns = ['Resoruces Use Indicators', 'A1 / A2 / A3', 'A4 Transport', 'A5 Installation', 'B1 Use', 'B2 Maintenance', 'B3 Repair', 'B4 Replacement', 'B5 Refurbishment', 'B6 Operational energy use', 'B7 Operational water use', 'C1 Deconstruction / demolition', 'C2 Transport', 'C3 Waste processing', 'C4 Disposal', 'D Reuse, recovery, recycling']
resource_use_start_keyword = "Use of renewable primary energy"
resource_use_num_columns = len(resource_use_columns)

# Parameters for Waste Output Flows
waste_pair1 = ("waste", "category")
waste_pair2 = ("hazardous", "waste")
waste_columns = ['Waste Category & Output Flows', 'A1 / A2 / A3', 'A4 Transport', 'A5 Installation', 'B1 Use', 'B2 Maintenance', 'B3 Repair', 'B4 Replacement', 'B5 Refurbishment', 'B6 Operational energy use', 'B7 Operational water use', 'C1 Deconstruction / demolition', 'C2 Transport', 'C3 Waste processing', 'C4 Disposal', 'D Reuse, recovery, recycling']
waste_start_keyword = "Hazardous waste disposed"
waste_num_columns = len(waste_columns)

In [26]:
#Extract the variations of keywords for extracting metadata (other_details)

file_path = r"C:\Users\S2878088\OneDrive - Saint-Gobain\Downloads\SMART\Dictionary.xlsx" 
df = pd.read_excel(file_path, sheet_name=1)

# Convert each column to a list including the header
data_with_headers = [[col] + df[col].tolist() for col in df.columns]

standard_details = [[x for x in sublist if not (pd.isna(x) or (isinstance(x, float) and math.isnan(x)))] for sublist in data_with_headers]

# Output the result
print(standard_details)

[['Other Details'], ['Valid Until:', 'Date of validity:', 'Valid until:'], ['Date of publication:', 'Date of issue:', 'Publication Date:', 'Original issue:'], ['FUNCTIONAL UNIT', 'DECLARED UNIT'], ['Scope of the EPD'], ['Third party verifier', 'Third-party verifier'], ['SYSTEM '], ['REFERENCE '], ['CUT-OFF RULES'], ['EPD Type'], ['Standards'], ['Site of manufacture', 'Production plant'], ['Management system', 'Management system', 'Owner', 'Programme used'], ['Product Name']]


In [22]:
#Extract the table as a DataFrame. 
#Input: The file path, keyword pairs to locate the table page, column names, the keyword to locate the start of the table, the number of columns, list of different variations of keywords of that particular table, list of lists different variations of keywords of all tables.
#Output: The particular table as a DataFrame
def extract_data_from_pdf(file_path, pair1, pair2, column_names, start_keyword, num_columns, ind_rows, standard_rows):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        temp_text = page.get_text()
        pair1_found, pair2_found = find_word_pairs(temp_text.split(), pair1, pair2)
        if pair1_found and pair2_found:
            text = temp_text
            break
            
    if not text:
        raise ValueError("Text with the required pairs not found in the PDF.")
    
    lines = extract_text_from_page(document, page_num, start_keyword)

    lines = [lines]
    
    lines = merge_numericals(lines)
    
    lines = attach_single_digit(lines)
    
    result = parse_lines_to_result(lines[0], ind_rows, standard_rows)
    
    tot_elements = sum(len(row) for row in result)
    
    if start_keyword.startswith("Climate Change"):
        no_elements = 208
    elif start_keyword.startswith("Use of renewable primary energy"):
        no_elements = 160
    else:
        no_elements = 128
        
    remaining = no_elements - tot_elements
    
    if remaining > 0:
        t_lines = extract_text_from_page(document, page_num + 1, start_keyword)
        t_lines = [t_lines]
        
        t_result = parse_lines_to_result(t_lines[0], ind_rows, standard_rows)
        result += t_result
    return pd.DataFrame(result, columns=column_names)

In [23]:
#Extract metadata as a dictionary (Everything other than the tables). 
#Input: The file path
#Output: Dictionary of metadata details
def extract_other_details_from_pdf(file_path):
    document = fitz.open(file_path)
    
#----- Scope -----
    
    page = document.load_page(0)
    text = page.get_text()
    sections = re.split(r'\n\n+', text.strip())
    lines = sections[0].split('\n')

    pos = next((i for i, line in enumerate(lines) if line.startswith("Scope of the EPD")), None)
    if pos is not None:
        scope_lines = lines[pos:]

    scope_element = next((item for item in lines if item.startswith('Scope of the EPD')), None)
    
    if scope_element:
        # Remove the 'Scope of the EPD' part
        scope = re.sub(r'Scope of the EPD(?:®)?: ', '', scope_element).strip()
        
        if scope == "" or scope == " ":
            scope = scope_lines[1]
            
        # If 'scope' ends with 'and', find the next word in the lines list and append it
        elif scope.endswith('and'):
            # Find the index of the scope_element in lines
            scope_index = lines.index(scope_element)
            
            # Check if there is a next line and append it to the scope
            if scope_index + 1 < len(lines):
                next_word = lines[scope_index + 1].strip()
                scope = f"{scope} {next_word}"

    else:
        scope = "Not found"


#----- Product Name -----
    
    if scope.startswith("Europe"):
        prod_name = extract_values_between_keywords(lines, 'EN 15804:2012+A2:2019 for:', 'Version')
        prod_name = add_orea_to_thickness(prod_name)
        if prod_name == '':
            prod_name = lines[0]+"Orae "+lines[-1]

    else:
        pattern = re.compile(r'EN 15804:2012\+A2:2019(?:/AC[: ]?2021)?')
    
        # Get starting index
        start_index = next((i for i, s in enumerate(lines) if pattern.search(s)), -1) + 1
        end_index = next(
        (index for index, item in enumerate(lines) if item.startswith('Version') or item.startswith('Date') or item.startswith('Manufacturer') or item.startswith('VERSION')),
        None)
        
        if start_index is not None and end_index is not None:
            prod_name = ''.join(lines[start_index:end_index]).strip()
        else:
            prod_name = "Not found"

    
#----- Valid Until -----

    #Load the Valid Until Keywords from the master excel file in sheet 2 
    valid_until_phrases = standard_details[1]
    
    valid_until_element = next(
        (item for item in lines if any(item.startswith(phrase) for phrase in valid_until_phrases)), 
        None )
    
    valid_until = valid_until_element.split(":", 1)[1].strip()
    
    if valid_until == "" or valid_until == " ":
        #Valid_until element is split from description, ex: "Valid Unitl:", "20-04-30" 
        if valid_until_element is not None:
            index = lines.index(valid_until_element)
            
            # Ensure there's an element after the found element
            if index + 1 < len(lines):
                valid_until = lines[index + 1]
            else:
                valid_until = "Not Found"
        else:
            valid_until = "Not Found"
    

#----- Date of Publication -----
    
    date_of_publication_phrases = standard_details[2]
    
    date_of_publication_element = next(
        (item for item in lines if any(item.startswith(phrase) for phrase in date_of_publication_phrases)), 
        None )

    
    # Remove the prefix and strip whitespace if the element is found
    if date_of_publication_element:
        date_of_publication = date_of_publication_element.split(":", 1)[1].strip()
    else:
        date_of_publication = "Not found"


    if date_of_publication == "" or date_of_publication == " ":
        if date_of_publication_element is not None:
            index = lines.index(date_of_publication_element)
            
            # Ensure there's an element after the found element
            if index + 1 < len(lines):
                date_of_publication = lines[index + 1]
            else:
                date_of_publication = "Not found"
        else:
            date_of_publication = "Not found"

    
#----- Third Party-----    
    
    verfiy_name1 = ("Geographical", "scope")
    verify_name2 = ("EPD®", "registration")
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        temp_text = page.get_text()
        pair1_found, pair2_found = find_word_pairs(temp_text.split(), verfiy_name1, verify_name2)
        if pair1_found and pair2_found:
            text = temp_text
            break
    
    sections = re.split(r'\n\n+', text.strip())
    lines = sections[0].split('\n')
    
    third_party_phrases = standard_details[5]
    # Check if any element contains "Third party verifier"
    third_party = any(phrase in item for item in lines for phrase in third_party_phrases)
    
    if third_party:
        third_party = "Verified"
    else:
        third_party = "Not Verified"


#----- Site of Manufacture -----
    
    site_keyword_start = standard_details[-3]
    site_keyword_end = standard_details[-2]
    
    start_index = next((i for i, item in enumerate(lines) if any(item.startswith(keyword) for keyword in site_keyword_start)), None)
    rem_lines = lines[start_index:]
    end_index = next((i for i, item in enumerate(rem_lines) if any(item.startswith(keyword) for keyword in site_keyword_end)), None)
 

    if start_index is not None and end_index is not None:
        production_plant = ' '.join(rem_lines[0:end_index]).strip()
        production_plant = production_plant.split(':', 1)[1].strip()
    else:
        production_plant = "Not found"

    
#----- Functional Unit -----
    
    functional_unit_pair = ("SYSTEM", "BOUNDARIES")
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        temp_text = page.get_text()
        pair1_found = find_word(temp_text.split(), functional_unit_pair)
        if pair1_found:
            text = temp_text
            break
    
    sections = re.split(r'\n\n+', text.strip())
    lines = sections[0].split('\n')
    normalized_data = [item.strip() for item in lines]

    functional_unit_phrases = standard_details[3]
    system_boundary_phrases = standard_details[6]
    reference_service_life_phrases = standard_details[7]
    cut_off_phrases = standard_details[8]

   
    # Extract Functional Unit
    start_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in functional_unit_phrases)), None)
    end_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in  system_boundary_phrases)), None)
    
    # Join lines between start_index and end_index if both are found
    if start_index is not None and end_index is not None:
        
        functional_unit = ' '.join(normalized_data[start_index:end_index])

        # Remove each phrase from functional_unit
        for phrase in functional_unit_phrases:
            functional_unit = functional_unit.replace(phrase, '')
        
        # Strip any extra spaces after replacements
        functional_unit = functional_unit.strip()
        
    else:
        
        functional_unit = "Not found"
    
    
    # Extract System Boundaries
    start_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in system_boundary_phrases)), None)
    end_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in  reference_service_life_phrases)), None)
    system_boundaries = ' '.join(item.strip() for item in lines[start_index + 1:end_index]) if start_index and end_index else "Not found"
    system_boundaries = system_boundaries.replace('SYSTEM', '').replace('BOUNDARIES', '').strip()
    
    # Extract Reference Service Life
    start_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in reference_service_life_phrases)), None)
    end_index = next((i for i, item in enumerate(normalized_data) if any(item.startswith(phrase) for phrase in  cut_off_phrases)), None)
    rsl = ' '.join(item.strip() for item in lines[start_index + 1:end_index]) if start_index and end_index else "Not found"
    match = re.search(r'(\d+)\s+years', rsl)
    rsl = match.group(1) if match else "Not found"
    
    
    return  {
        "EPD Type": "Product Specific",
        "Third Party": third_party,
        "Geographical Scope": scope,
        "Standards": "EN 15804 and ISO 21930",
        "life_cycle_stages" : ["A1 / A2 / A3", "A4 Transport", "A5 Installation", "B1 Use", "B2 Maintenance", "B3 Repair", "B4 Replacement", "B5 Refurbishment", "B6 Operational energy use", "B7 Operational water use", "C1 Deconstruction / demolition", "C2 Transport", "C3 Waste processing", "C4 Disposal", "D Reuse, recovery, recycling"],
        "Functional Unit": functional_unit,
        "Reference Service Life": rsl,
        "System Boundaries": system_boundaries,
        "Date of Publication": date_of_publication,
        "Site of Manufacture": production_plant,
        "Product Name": prod_name,
        "Valid Until": valid_until,
    }
   

In [ ]:
#Process (Concatenate all tables, add metadata as columns and do all the data formatting) all the data together and store it in an 
#  file.
#Input: The three tables as DataFrame, dictionary containing metadata detials and name of the pdf. 
#Output: Store the data of EPD (the tables and metadata) in an excel in the 'Excel' directory.
def process_and_save_data(env_impacts_df, resource_use_df, waste_impacts_df, other_details, pdf, output_path='Excel'):
    # Update column names for each DataFrame
    env_impacts_df.columns = ['Indicators'] + env_impacts_df.columns[1:].tolist()
    resource_use_df.columns = ['Indicators'] + resource_use_df.columns[1:].tolist()
    waste_impacts_df.columns = ['Indicators'] + waste_impacts_df.columns[1:].tolist()

    # Concatenate the DataFrames vertically
    combined_df = pd.concat([env_impacts_df, resource_use_df, waste_impacts_df], axis=0)


    # Add each detail as a new column to combined_df, filled with the corresponding value
    for detail, value in other_details.items():
        combined_df[detail] = [value] * len(combined_df)

   # Parse dates with multiple formats
    combined_df['Date of Publication'] = pd.to_datetime(combined_df['Date of Publication'], errors='coerce', dayfirst=True)
    combined_df['Valid Until'] = pd.to_datetime(combined_df['Valid Until'], errors='coerce', dayfirst=True)
    
    # Convert to DD-MM-YYYY format
    combined_df['Date of Publication'] = combined_df['Date of Publication'].dt.strftime('%d-%m-%Y')
    combined_df['Valid Until'] = combined_df['Valid Until'].dt.strftime('%d-%m-%Y')
    
    # List of columns to be converted to floating-point numbers
    columns_to_convert = ['A1 / A2 / A3', 'A4 Transport', 'A5 Installation', 'B1 Use', 'B2 Maintenance',
                          'B3 Repair', 'B4 Replacement', 'B5 Refurbishment', 'B6 Operational energy use', 
                          'B7 Operational water use', 'C1 Deconstruction / demolition', 'C2 Transport', 
                          'C3 Waste processing', 'C4 Disposal', 'D Reuse, recovery, recycling']

    
    def clean_and_convert_to_float(series):
        # Replace commas with periods
        series = series.str.replace(',', '.', regex=False)
        # Convert to numeric (float), coercing errors to NaN
        return pd.to_numeric(series, errors='coerce')
    
    # Apply the cleaning and conversion function to each column
    combined_df[columns_to_convert] = combined_df[columns_to_convert].apply(clean_and_convert_to_float)


    # Save the data to Excel
    file_path = f'{output_path}/{pdf}_data.xlsx'
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        combined_df.to_excel(writer, sheet_name='Combined Data_check', index=False)

    print(f'Data successfully saved to {file_path}')

In [27]:
# # Extract data of all available EPD from the 'EPD' directory. 

# # Glass (29 EPD's)
# for i in range(1,30):
#     pdf = f"Glass{i}"
#     pdf_path = f"EPD/{pdf}"
#     env_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", env_impacts_pair1, env_impacts_pair2, env_impacts_columns, env_impacts_start_keyword, env_impacts_num_columns, env_rows, standard_rows)
#     resource_use_df = extract_data_from_pdf(f"{pdf_path}.pdf", resource_use_pair1, resource_use_pair2, resource_use_columns, resource_use_start_keyword, resource_use_num_columns, resource_rows, standard_rows)
#     waste_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", waste_pair1, waste_pair2, waste_columns, waste_start_keyword, waste_num_columns, waste_rows, standard_rows)
#     other_details = extract_other_details_from_pdf(f"{pdf_path}.pdf")
#     process_and_save_data(env_impacts_df, resource_use_df, waste_impacts_df, other_details, pdf, output_path='Excel')

# # Gyproc (10 EPD's)
# for i in range(1,12):
#     if i == 9:
# # This EPD is not valid
#         continue
#     pdf = f"Gyproc{i}"
#     pdf_path = f"EPD/{pdf}"
#     env_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", env_impacts_pair1, env_impacts_pair2, env_impacts_columns, env_impacts_start_keyword, env_impacts_num_columns, env_rows, standard_rows)
#     resource_use_df = extract_data_from_pdf(f"{pdf_path}.pdf", resource_use_pair1, resource_use_pair2, resource_use_columns, resource_use_start_keyword, resource_use_num_columns, resource_rows, standard_rows)
#     waste_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", waste_pair1, waste_pair2, waste_columns, waste_start_keyword, waste_num_columns, waste_rows, standard_rows)
#     other_details = extract_other_details_from_pdf(f"{pdf_path}.pdf")
#     process_and_save_data(env_impacts_df, resource_use_df, waste_impacts_df, other_details, pdf, output_path='Excel')

# # VDS (3 EPD's)
# for i in range(1,4):
#     pdf = f"VDS{i}"
#     pdf_path = f"EPD/{pdf}"
#     env_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", env_impacts_pair1, env_impacts_pair2, env_impacts_columns, env_impacts_start_keyword, env_impacts_num_columns, env_rows, standard_rows)
#     resource_use_df = extract_data_from_pdf(f"{pdf_path}.pdf", resource_use_pair1, resource_use_pair2, resource_use_columns, resource_use_start_keyword, resource_use_num_columns, resource_rows, standard_rows)
#     waste_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", waste_pair1, waste_pair2, waste_columns, waste_start_keyword, waste_num_columns, waste_rows, standard_rows)
#     other_details = extract_other_details_from_pdf(f"{pdf_path}.pdf")
#     process_and_save_data(env_impacts_df, resource_use_df, waste_impacts_df, other_details, pdf, output_path='Excel')


# # Provide the name of EPD in 'pdf' variable. The EPD must be stored in 'EPD' directory. 
# # Extracts all the tables and metadata and finally processes and stores it as an excel in 'Excel' directory. 
pdf = f"Glass1"
pdf_path = f"EPD/{pdf}"
env_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", env_impacts_pair1, env_impacts_pair2, env_impacts_columns, env_impacts_start_keyword, env_impacts_num_columns, env_rows, standard_rows)
resource_use_df = extract_data_from_pdf(f"{pdf_path}.pdf", resource_use_pair1, resource_use_pair2, resource_use_columns, resource_use_start_keyword, resource_use_num_columns, resource_rows, standard_rows)
waste_impacts_df = extract_data_from_pdf(f"{pdf_path}.pdf", waste_pair1, waste_pair2, waste_columns, waste_start_keyword, waste_num_columns, waste_rows, standard_rows)
other_details = extract_other_details_from_pdf(f"{pdf_path}.pdf")
process_and_save_data(env_impacts_df, resource_use_df, waste_impacts_df, other_details, pdf, output_path='Excel')

Data successfully saved to Excel/Glass1_data.xlsx


C:\Users\S2878088\AppData\Local\Temp\ipykernel_14364\3972470706.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  combined_df['Date of Publication'] = pd.to_datetime(combined_df['Date of Publication'], errors='coerce', dayfirst=True)
C:\Users\S2878088\AppData\Local\Temp\ipykernel_14364\3972470706.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  combined_df['Valid Until'] = pd.to_datetime(combined_df['Valid Until'], errors='coerce', dayfirst=True)
